In [0]:
!pip3 install torch
!pip3 install skorch
from skorch import NeuralNetClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time
import pathlib
import dill
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
preprocessed_folder = pathlib.Path('/content/drive/My Drive/validated_clips/preprocessed/')
mfccs_padded = {"dev":[], "train":[], "test":[]}
labels={"dev":[], "train":[], "test":[]}
for i in range(1,20):
  number = '{:.6f}'.format(i)
  mfcc_filename = number+"mfccs_padded.pkl"
  label_filename = number+"label.pkl"
  mfcc_filepath = preprocessed_folder / mfcc_filename
  label_filepath = preprocessed_folder / label_filename

  with open(mfcc_filepath, "rb") as f:
    mfcc_local = dill.load(f)
  mfccs_padded['dev'].extend(mfcc_local['dev'])
  mfccs_padded['train'].extend(mfcc_local['train'])
  mfccs_padded['test'].extend(mfcc_local['test'])
  
  with open(label_filepath, "rb") as f:
    label_local = dill.load(f)
  labels['dev'].extend(label_local['dev'])
  labels['train'].extend(label_local['train'])
  labels['test'].extend(label_local['test'])

print(len(mfccs_padded))
print(len(labels))

3
3


In [0]:
print(type(mfccs_padded))
print(len(mfccs_padded["train"]))
print(type(mfccs_padded["train"]))
print(type(mfccs_padded["train"][0]))
print(mfccs_padded["train"][0].shape)
print([len(array) for array in mfccs_padded["train"]])

<class 'dict'>
199
<class 'list'>
<class 'numpy.ndarray'>
(2440, 13)
[2440, 2440, 2440, 2440, 2440, 2440, 2440, 2440, 2440, 2440, 2440, 2440, 42760, 42760, 42760, 42760, 6234, 1986, 1986, 1986, 1986, 1986, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2209, 2569, 2569, 2569, 3282, 3282, 3282, 3282, 3282, 3282, 3282, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2944, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2721, 2310, 2310, 2310, 2310, 2310, 2310, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2584, 2195, 2195, 2195, 2195, 2195, 2195, 2195, 2454, 2454, 2454, 2454, 2886, 2886, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 3354, 22

In [0]:
mfccs_padded = {key:np.stack(value) for key,value in mfccs_padded.items()}
print(mfccs_padded['train'].shape)

ValueError: ignored

In [0]:
class downwardSlope(nn.Module):
    def __init__(self, maxSeqLength, outsize):
        super().__init__()
        self.maxSeqLength = maxSeqLength

        self.conv1 = nn.Conv1d(13, 32, 15, padding=1)
        self.conv2 = nn.Conv1d(32, 32, 15, padding=1)
        self.conv3 = nn.Conv1d(32, 32, 15, padding=1)
        self.fc1 = nn.Linear(2908, outsize)

    def forward(self,x):
        print("insideforward",type(x), x.dtype)
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return torch.squeeze(x)

NameError: ignored

In [0]:
def evaluateModel(model, lossCriterion, X, y, batchSize):
    with torch.no_grad():
        n=batchSize
        batched = list(zip([X[i:i + n] for i in range(0, len(X), n)],
                          [y[i:i + n] for i in range(0, len(y), n)]))
        predList = []
        yList = []
        lossMeans = []
        for i,(batchX, batchY) in enumerate(batched):
            print('x',type(X), X.dtype)
            print('batchx',type(batchX), batchX.dtype)
            predProb = model(batchX)
            if np.isnan(predProb.cpu().detach().numpy()).any():
                print("AAAAAA a NAN")
                return
            loss = lossCriterion(predProb, batchY)
            batchPred = predProb.round().tolist()
            predList = predList+batchPred
            batchYargmaxed = batchY.tolist()
            yList = yList+batchYargmaxed
            lossMeans.append(torch.mean(loss).item())

        #acc = accuracy_score(yList, predList)
        #report = classification_report(yList, predList)
        #confMat = confusion_matrix(yList, predList)
        lossMean = sum(lossMeans)/float(len(lossMeans))

    return lossMean

In [0]:
def trainModel(model, optimizer, criterion, train_X, train_y, val_X, val_y, batchSize, startEpoch, endEpoch):
    notifyEvery = 100 if torch.cuda.is_available() else 2

    checkmarkTime = time.time()

    n=batchSize
    batched = list(zip([train_X[i:i + n] for i in range(0, len(train_X), n)],
                      [train_y[i:i + n] for i in range(0, len(train_y), n)]))
    numBatches = len(batched)
    print("number of batches", numBatches)
    for epoch in range(startEpoch, endEpoch):
        print("epoch:", epoch)
        for i,(batchX,batchy) in enumerate(batched):
            optimizer.zero_grad()
            output =  model(batchX)
            loss = criterion(output, batchy)
            loss.backward()
            optimizer.step()
            if np.isnan(output.cpu().detach().numpy()).any():
                print("AAAAAA a NAN")
                return
            if i%notifyEvery ==notifyEvery-1:
                print('[%d, %5d]' %
                  (epoch + 1, i + 1))
                timeTook = time.time() - checkmarkTime
                print("took", timeTook, "seconds for", notifyEvery, "batches")
                if(torch.cuda.is_available()):
                    print(torch.cuda.max_memory_allocated()/1e9, "GB of VRAM being used")
                checkmarkTime = time.time()
        trainLoss = evaluateModel(model,criterion,train_X, train_y, batchSize)
        valLoss = evaluateModel(model, criterion,val_X, val_y, batchSize)
        print('loss', valLoss)
    print("finished training")

In [0]:
X = torch.as_tensor(mfccs_padded, dtype=torch.float)
le = preprocessing.LabelEncoder()
le.fit(labels)
transformed = le.transform(labels)
print(type(transformed),transformed.shape, print(transformed))
enc =  preprocessing.OneHotEncoder(handle_unknown='ignore')
onehotted =enc.fit_transform(transformed.reshape(-1,1)).toarray() 
optimizer = torch.optim.Adam(mod.parameters(), lr = learningRate)

y = torch.as_tensor(onehotted, dtype=torch.long)
outsize = len(le.classes_)
print(outsize, type(onehotted), type(y), onehotted.shape)

device = "cpu"
seqLength = mfccs_padded.shape[2]
mod = downwardSlope(seqLength, outsize)
criterion = nn.CrossEntropyLoss()
endEpoch = 3
learningRate = 0.001
print('x',type(X), X.dtype)
trainModel(mod, optimizer,criterion, X[:20], y[:20],X[20:],y[20:], 10, 0, 10)

NameError: ignored